In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import operator
import itertools
import scipy.stats as stats

In [7]:
df = pd.read_csv('puma8.csv')
df['thetadd3_bin'] = 1
df.loc[df['thetadd3'] < 1, 'thetadd3_bin'] = 0
Y_all = df[['thetadd3_bin']]
X_all = df.drop(['thetadd3', 'thetadd3_bin'], axis=1)
X_train_ori, X_test_ori, y_train, y_test = train_test_split(X_all, Y_all, test_size = 0.3, random_state = 0)


In [9]:
def numpy_discretize(X_train, X_test, gran=10):
    """
    multi-granularity discretization
    method. The basic idea is simple: instead of using a fine-tuned
    granularity, we discretize each numerical feature into several, rather
    than only one, categorical features, each with a different granularity.
    
    min granularity = 10
    
    Sometimes de edge values did not permit to execute correct discretization
    if this happens the step is not executed
    """
    
    # separa dados numericos que precisam de binarizacao
    is_numeric = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric_features = X_train.select_dtypes(include=is_numeric)
    numeric_features = [feat for feat in numeric_features.columns if 'disc' not in feat[-4:]]
    discrete_features = []
#     print(f"Discretizing {len(numeric_features.columns)} features...")
    feat_count = 0
    for feat in numeric_features:
        this_gran = gran
        if feat_count % 50 == 0:
            print(f" Working in {feat}")
        X_train_np = X_train[[feat]].to_numpy()
        X_test_np = X_test[[feat]].to_numpy()
        success = False
        while not success:
            try:
                D_train = np.zeros([X_train.shape[0], 1])
                D_test = np.zeros([X_test.shape[0], 1])
                # calc numpy histogram and apply to features
                hist, bin_edges = np.histogram(X_train_np[:, 0], bins=this_gran)
                D_train[:, 0] = np.digitize(X_train_np[:,0], bin_edges, right=False)
                D_test[:, 0] = np.digitize(X_test_np[:,0], bin_edges, right=False)

                # apply back to pandas
                X_train[f"{feat}_disc"] = D_train
                X_test[f"{feat}_disc"] = D_test
                success = True
            except:
                print(f"Not possible to correct work on cut {feat} > {this_gran}")
                this_gran = this_gran - 1
                if this_gran <= 1:
                    success = True
        
        feat_count += 1
    return X_train, X_test



def min_max_scaler(X_train, X_test):
    scaler = MinMaxScaler()
    scaler.fit(X_train[ORIGINAL_FEATURES])
    norm_feats = [f"{x}_norm" for x in ORIGINAL_FEATURES if 'disc' not in x[-4:]]
    X_train = X_train.reindex(columns=X_train.columns.tolist() + norm_feats)
    X_test = X_test.reindex(columns=X_test.columns.tolist() + norm_feats)
    X_train.loc[:, norm_feats] = scaler.transform(X_train[ORIGINAL_FEATURES])
    X_test.loc[:, norm_feats] = scaler.transform(X_test[ORIGINAL_FEATURES])
    return X_train, X_test


def binary_operators(df):
    # calc all pair columns
    all_columns = list(df)
    pairwise_cols = list(itertools.combinations(all_columns, 2))
    for pair in pairwise_cols:
        df[f"{pair[0]}_x_{pair[1]}_op_sum"] = df[pair[0]] + X_train[pair[1]]
        df[f"{pair[0]}_x_{pair[1]}_op_sub"] = df[pair[0]] - X_train[pair[1]]
        df[f"{pair[0]}_x_{pair[1]}_op_mul"] = df[pair[0]] * X_train[pair[1]]
        df[f"{pair[0]}_x_{pair[1]}_op_div"] = df[pair[0]] / X_train[pair[1]]
        df = df.replace([np.inf, -np.inf], np.nan)
    
    return df


def high_order_operators(df):
    group_columns = [col for col in df.columns if "disc" in col]
    to_group_columns = [col for col in df.columns if "disc" not in col]
    all_dfs = pd.DataFrame()
    for col in group_columns:
        print(f"Grouping {col}")
        df_avg = df[to_group_columns+[col]].groupby(col).transform('mean').add_suffix(f'_group_by_{col}_and_mean')
        df_min = df[to_group_columns+[col]].groupby(col).transform('min').add_suffix(f'_group_by_{col}_and_min')
        df_max = df[to_group_columns+[col]].groupby(col).transform('max').add_suffix(f'_group_by_{col}_and_max')
        all_dfs = pd.concat([all_dfs, df_avg, df_min, df_max], axis=1,  sort=False)
    all_dfs = pd.concat([df,all_dfs], axis=1)
    return all_dfs
        

In [10]:
ORIGINAL_FEATURES = X_train_ori.columns

# discretize
X_train, X_test = numpy_discretize(X_train_ori.copy(), X_test_ori.copy(), gran=10)
# normalize
X_train, X_test = min_max_scaler(X_train.copy(), X_test.copy())

step1_train = X_train.copy()
step1_test = X_test.copy()
step1_features = X_train.columns

 Working in theta1


In [12]:
X_train = high_order_operators(X_train[step1_features].copy())
X_test = high_order_operators(X_test[step1_features].copy())

step2_train = X_train.copy()
step2_test = X_test.copy()
step2_features = X_train.columns

Grouping theta1_disc
Grouping theta2_disc
Grouping theta3_disc
Grouping thetad1_disc
Grouping thetad2_disc
Grouping thetad3_disc
Grouping tau1_disc
Grouping tau2_disc
Grouping theta1_disc
Grouping theta2_disc
Grouping theta3_disc
Grouping thetad1_disc
Grouping thetad2_disc
Grouping thetad3_disc
Grouping tau1_disc
Grouping tau2_disc


In [13]:
# binary operators
X_train = binary_operators(X_train[step1_features].copy())
X_test = binary_operators(X_test[step1_features].copy())

step3_train = X_train.copy()
step3_test = X_test.copy()
step3_features = X_train.columns

In [14]:
X_train = pd.concat([step1_train, step2_train, step3_train], axis=1).dropna(axis=1, thresh=0.03*X_train.shape[0])
X_test = pd.concat([step1_test, step2_test, step3_test], axis=1)[X_train.columns]

In [15]:
# discretize
X_train, X_test = numpy_discretize(X_train, X_test, gran=10)


 Working in theta1
 Working in theta3_group_by_theta1_disc_and_min
 Working in thetad2_group_by_theta2_disc_and_min
 Working in tau1_group_by_theta3_disc_and_min
 Working in theta1_norm_group_by_thetad1_disc_and_min
 Working in theta3_norm_group_by_thetad2_disc_and_min
 Working in thetad2_norm_group_by_thetad3_disc_and_min
 Working in tau1_norm_group_by_tau1_disc_and_min
 Working in theta1_group_by_tau2_disc_and_max
 Working in theta1_x_thetad3_op_mul
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 10
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 9
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 8
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 7
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 6
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 5
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 4
Not possible to correct work on cut theta1_x_theta1_norm_op_div > 3

In [16]:
X_train

,theta1,theta2,theta3,thetad1,thetad2,thetad3,tau1,tau2,theta1_disc,theta2_disc,...,thetad2_norm_x_tau2_norm_op_mul_disc,thetad3_norm_x_tau1_norm_op_sum_disc,thetad3_norm_x_tau1_norm_op_sub_disc,thetad3_norm_x_tau1_norm_op_mul_disc,thetad3_norm_x_tau2_norm_op_sum_disc,thetad3_norm_x_tau2_norm_op_sub_disc,thetad3_norm_x_tau2_norm_op_mul_disc,tau1_norm_x_tau2_norm_op_sum_disc,tau1_norm_x_tau2_norm_op_sub_disc,tau1_norm_x_tau2_norm_op_mul_disc
7252,-0.994493,1.336377,-0.634258,1.327497,0.902959,-1.622991,0.467831,0.428653,3.0,9.0,...,7.0,5.0,1.0,1.0,5.0,1.0,1.0,9.0,6.0,8.0
238,-0.422692,-1.674322,-1.708412,-1.645784,-0.859870,-1.757971,-0.455449,0.075630,4.0,1.0,...,2.0,1.0,5.0,1.0,3.0,3.0,1.0,4.0,3.0,1.0
1524,0.243595,-0.015335,-1.383064,0.117840,1.337653,-1.820437,0.164156,0.176822,6.0,5.0,...,6.0,4.0,2.0,1.0,4.0,2.0,1.0,7.0,5.0,5.0
4827,1.118728,-1.848653,1.303732,1.104385,-1.647377,-1.243248,0.426595,-0.219039,8.0,1.0,...,1.0,6.0,2.0,2.0,3.0,5.0,1.0,6.0,8.0,3.0
2165,0.652088,0.518338,-0.555735,1.604987,1.299452,0.790726,0.254276,0.414478,7.0,7.0,...,8.0,8.0,5.0,6.0,8.0,5.0,7.0,8.0,5.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,-0.192198,0.536369,0.904232,0.921560,1.856092,-0.674148,-0.579422,-0.337847,5.0,7.0,...,3.0,2.0,7.0,1.0,3.0,6.0,1.0,2.0,4.0,1.0
3264,-1.640719,1.331985,0.945258,0.548063,1.400007,-1.871477,-0.494007,0.022944,1.0,9.0,...,5.0,1.0,5.0,1.0,3.0,3.0,1.0,3.0,3.0,1.0
1653,1.149387,0.729353,-1.514101,-0.588988,0.661809,1.782106,-0.027575,-0.254940,9.0,7.0,...,2.0,8.0,8.0,5.0,7.0,9.0,3.0,4.0,6.0,2.0
2607,1.684261,-0.889879,-0.956487,-0.779190,0.825917,0.368874,-0.585570,-0.469307,10.0,3.0,...,1.0,4.0,8.0,1.0,4.0,8.0,1.0,1.0,5.0,1.0


In [34]:
def dataset_based_meta_features(X_train, X_test, y_train, y_test):
    """
    Applied in the original set!
    """
    
    
    
    def _general_information(X):
        dataset_info_df= pd.DataFrame()

        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        num_numeric_attr = X.select_dtypes(include=numerics).shape[1]
        num_duscrete_attr = X.shape[1] - num_numeric_attr
            
        
        dataset_info_df['num_instances'] = [X.shape[0]]
        dataset_info_df['num_features'] = X.shape[1]
        
        dataset_info_df['num_numeric_attr'] = num_numeric_attr
        dataset_info_df['num_discrete_attr'] = num_duscrete_attr
        dataset_info_df['ratio_numeric_attr'] = num_numeric_attr/ (num_numeric_attr+num_duscrete_attr)
        dataset_info_df['ratio_discrete_attr'] = num_duscrete_attr/ (num_numeric_attr+num_duscrete_attr)
        
        return dataset_info_df
    
    


    def _initial_evaluation(X_train, X_test, y_train, y_test):
    
        from sklearn.ensemble import RandomForestClassifier
        from sklearn import metrics
        
        def acc(y_true, y_pred):
            from sklearn.metrics import accuracy_score
            y_pred = list(map(lambda k: 0 if k<=0.5 else 1, y_pred))
            return accuracy_score(y_true, y_pred)

        def f1(y_true, y_pred, th):
            from sklearn.metrics import f1_score
            y_pred = list(map(lambda k: 0 if k<=th else 1, y_pred))
            return f1_score(y_true, y_pred)

        def precision(y_true, y_pred, th):
            from sklearn.metrics import precision_score
            y_pred = list(map(lambda k: 0 if k<=th else 1, y_pred))
            return precision_score(y_true, y_pred, average='macro') 

        def recall(y_true, y_pred, th):
            from sklearn.metrics import recall_score
            y_pred = list(map(lambda k: 0 if k<=th else 1, y_pred))
            return recall_score(y_true, y_pred, average='macro')

        def auc(y_true, y_pred):
            from sklearn.metrics import roc_auc_score
            return roc_auc_score(y_true, y_pred)


        df_initial_evaluation = pd.DataFrame()
        clf = RandomForestClassifier()
        clf.fit(X_train, y_train)
        y_pred = clf.predict_proba(X_test)[:,1]

        for th in [0.4, 0.45, 0.5, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]:
            df_initial_evaluation[f'f1_{th}'] = [f1(y_test, y_pred, th)]
            df_initial_evaluation[f'precision_{th}'] = precision(y_test, y_pred, th)
            df_initial_evaluation[f'recall_{th}'] = recall(y_test, y_pred, th)

        df_initial_evaluation['auc'] = auc(y_test, y_pred)
        
        df_initial_evaluation['avg_f1'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'f1' in col]].mean(axis=1)
        df_initial_evaluation['std_f1'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'f1' in col]].std(axis=1)
        df_initial_evaluation['max_f1'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'f1' in col]].max(axis=1)
        df_initial_evaluation['min_f1'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'f1' in col]].min(axis=1)
    
        df_initial_evaluation['avg_precision'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'precision' in col]].mean(axis=1)
        df_initial_evaluation['std_precision'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'precision' in col]].std(axis=1)
        df_initial_evaluation['max_precision'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'precision' in col]].max(axis=1)
        df_initial_evaluation['min_precision'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'precision' in col]].min(axis=1)
    
        df_initial_evaluation['avg_recall'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'recall' in col]].mean(axis=1)
        df_initial_evaluation['std_recall'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'recall' in col]].std(axis=1)
        df_initial_evaluation['max_recall'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'recall' in col]].max(axis=1)
        df_initial_evaluation['min_recall'] = df_initial_evaluation[[col for col in df_initial_evaluation.columns if 'recall' in col]].min(axis=1)
    
        return df_initial_evaluation
    
    
    def _entropy_based_measures(X_train, y_train):
        import pymit

        df_mutual_info = pd.DataFrame()

        for feat in X_train.columns:
            df_mutual_info[feat] = [pymit.I(X_train[feat].values, y_train['thetadd3_bin'].values , bins=[10,2])]
        
        df_mutual_info['avg_mi'] = df_mutual_info.mean(axis=1)
        df_mutual_info['std_mi'] = df_mutual_info.std(axis=1)
        df_mutual_info['min_mi'] = df_mutual_info.min(axis=1)
        df_mutual_info['max_mi'] = df_mutual_info.max(axis=1)
        return df_mutual_info[['avg_mi', 'std_mi', 'min_mi', 'max_mi']]
    
    
    def _feature_diversity(X_train):
        
        df_feature_diversity = pd.DataFrame()
        
        disc_columns = [col for col in X_train if 'disc' in col]
        numeric_columns = [col for col in X_train if 'disc' not in col]
        
        numeric_pairs = list(itertools.combinations(numeric_columns, 2))
        all_t = []
        for pair in numeric_pairs:
            t_pair = stats.ttest_rel(X_train[pair[0]].values, X_train[pair[1]].values)[0]
            all_t.append(t_pair)
        
        all_chi = []
        disc_pairs = list(itertools.combinations(disc_columns, 2))
        for pair in disc_pairs:
            contingency = pd.crosstab(X_train[pair[0]].values, X_train[pair[1]].values) 
            chi, _, _, _ = stats.chi2_contingency(contingency) 
            all_chi.append(chi)
        
        df_feature_diversity['avg_t'] = [np.mean(all_t)]
        df_feature_diversity['std_t'] = np.std(all_t)
        df_feature_diversity['max_t'] = np.max(all_t)
        df_feature_diversity['min_t'] = np.min(all_t)
        
        df_feature_diversity['avg_chi'] = np.mean(chi)
        df_feature_diversity['std_chi'] = np.std(chi)
        df_feature_diversity['max_chi'] = np.max(chi)
        df_feature_diversity['min_chi'] = np.min(chi)
        
        return df_feature_diversity
    
    dataset_info_df = _general_information(X_train.copy())
    dataset_initial_eval = _initial_evaluation(X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy())
    dataset_entropy_info = _entropy_based_measures(X_train.copy(), y_train.copy())
    dataset_feature_diversity = _feature_diversity(X_train.copy())

    
    df = pd.concat([dataset_info_df, dataset_initial_eval, dataset_entropy_info, dataset_feature_diversity], axis=1)
    return df
        
        
        

In [35]:
p = dataset_based_meta_features(X_train_ori.copy(), X_test_ori.copy(), y_train.copy(), y_test.copy())

/home/fernando.favoretti/anaconda3/envs/basic/lib/python3.7/site-packages/ipykernel_launcher.py:61: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [36]:
p

,num_instances,num_features,num_numeric_attr,num_discrete_attr,ratio_numeric_attr,ratio_discrete_attr,f1_0.4,precision_0.4,recall_0.4,f1_0.45,...,min_mi,max_mi,avg_t,std_t,max_t,min_t,avg_chi,std_chi,max_chi,min_chi
0,5734,16,16,0,1.0,0.0,0.821028,0.815713,0.811911,0.82797,...,0.000951,0.155866,0.387909,1.138732,2.111401,-1.844833,105.994616,0.0,105.994616,105.994616


In [112]:
x = X_train.head(10)['theta1'].values

In [126]:
p = X_train.tail(10)['theta1'].values

In [122]:
y = y_train['thetadd3_bin'].head(10).values

In [137]:
_entropy_based_measures(X_train_ori, y_train)

,theta1,theta2,theta3,thetad1,thetad2,thetad3,tau1,tau2
0,0.002284,0.155866,0.093736,0.001927,0.000951,0.001575,0.001431,0.001576


In [131]:
df_mutual_info

NameError: name 'df_mutual_info' is not defined

In [128]:
??pymit.I

Signature: pymit.I(X, Y, bins)
Source:   
def I(X,Y,bins):
    """
    Calculates the mutual information of X and Y.
    If X and Y are already discretised, set \ref bins to the amount of bins, aka states of X and Y.
    If X and Y are not discretised, \ref bins will be used, to diskretise X and Y into \ref bins states
    
    @param X numpy vector
    @param Y numpy vector
    @param bins If X and Y are already diskretised specify the amount of bins of X and Y. If X and Y are not discretised specifies the amount of bins to diskretise X and Y into
                bins can be spcified as tuple, e.g. bins = (bins_x, bins_y), to diskretise X and Y independently, or as a single value to diskretise X and Y similar, e.g. bins = amount_of_bins
    @return Returns the mutual information
    """
    if (type(bins) == list):
        ybins = bins[1]
        xbins = bins[0]
        base = LOG_BASE
    else:
        ybins = bins
        xbins = bins
        base = bins
    
    p_xy, _,_ = numpy.h

In [127]:
pymit.H_cond(x, p, bins=10)

0.18061799739838869